In [ ]:
import json
import requests

def match(conditions):
    return requests.post('http://localhost:5004/match', json=conditions).json()

def get_annotated_data():
    url = 'https://storage.gra.cloud.ovh.net/v1/AUTH_32c5d10cb0fe4519b957064a111717e3/models/match_pubmed_affiliations_with_countries_v3.json'
    data = requests.get(url).json()
    json.dump(data, open('match_pubmed_affiliations_with_countries_v3.json', 'w'), indent=2, ensure_ascii=False)
    return data

def compute_precision_recall(match_type, index_prefix=''):
    data = get_annotated_data()
    nb_TP, nb_FP, nb_FN = 0, 0, 0
    false_positive, false_negative = [], []
    for ix, d in enumerate(data):
        if ix%100==0:
            print(ix, end=',')
        if d.get(match_type):
            res = match({'query': d['label'], 'year': '2020', 'type': match_type, 'index_prefix': index_prefix})
            for x in res['results']:
                if x in d[match_type]:
                    nb_TP += 1
                else:
                    nb_FP += 1
                    false_positive.append(d)
            for x in d[match_type]:
                if x not in res['results']:
                    nb_FN += 1
                    false_negative.append(d)
    precision = nb_TP / (nb_TP + nb_FP)
    recall = nb_TP / (nb_TP + nb_FN)
    res = {'precision' : precision, 'recall' : recall}
    return {'res': res, 'false_positive': false_positive, 'false_negative': false_negative}

In [ ]:
metrics_country = compute_precision_recall(match_type='country', index_prefix='matcher')
metrics_country['res']

In [ ]:
metrics_country['false_positive']

In [ ]:
metrics_grid = compute_precision_recall(match_type='grid', index_prefix='matcher')
metrics_grid['res']

In [ ]:
metrics_grid['false_positive']

In [ ]:
metrics_rnsr = compute_precision_recall(match_type='rnsr', index_prefix='matcher')
metrics_rnsr['res']

In [ ]:
metrics_rnsr['false_positive']

In [ ]:
# No ROR data into the reference file
# metrics_ror = compute_precision_recall(match_type='ror', index_prefix='matcher')
# metrics_ror['res']

In [ ]:
# metrics_ror['false_positive']